## Part of the class ***Dataloader*** source code is cited below

In [ ]:
# 以下为dataloader类的官方源码的一部分
class DataLoader(Generic[T_co]):
    dataset: Dataset[T_co]      # 此参数是上面自定义的dataset类所实例化的对象
    batch_size: Optional[int]   
    num_workers: int            # 设置几个进程加载数据
    pin_memory: bool
    drop_last: bool
    timeout: float
    sampler: Sampler            # 如需输入的训练样本的顺序有特定需求，可通过自定义sampler或batch sampler实现（nlp中常用）
    prefetch_factor: int
    _iterator : Optional['_BaseDataLoaderIter']
    __initialized = False

    def __init__(self, dataset: Dataset[T_co], batch_size: Optional[int] = 1,
                 shuffle: bool = False, sampler: Optional[Sampler] = None,  # shuffle可以在每个epoch后打乱数据集，自定义sampler时，shuffle无意义
                 batch_sampler: Optional[Sampler[Sequence]] = None,          
                 num_workers: int = 0, collate_fn: Optional[_collate_fn_t] = None, # collate_fn对一个batch再处理，如对batch进行pad等（nlp中常用）
                 pin_memory: bool = False, drop_last: bool = False,
                 timeout: float = 0, worker_init_fn: Optional[_worker_init_fn_t] = None,
                 multiprocessing_context=None, generator=None,
                 *, prefetch_factor: int = 2,
                 persistent_workers: bool = False):       
        
        ...... # 后面内容省略
        
        
        
        if sampler is not None and shuffle:
            raise ValueError('sampler option is mutually exclusive with '
                             'shuffle')  # sampler和shuffle一起设置是矛盾的
                
        if batch_sampler is not None:
            # auto_collation with custom batch_sampler
            if batch_size != 1 or shuffle or sampler is not None or drop_last:
                raise ValueError('batch_sampler option is mutually exclusive '
                                 'with batch_size, shuffle, sampler, and '
                                 'drop_last') # 如果设置了batch_sampler，则再去设置batchsize，shuffle，sampler或drop_last都与之矛盾
                
            ...... # 后面内容省略
                
                
        if sampler is None:  # give default samplers 
            if self._dataset_kind == _DatasetKind.Iterable:   # 如果dataset是iterable类型的（一般不用）
                # See NOTE [ Custom Samplers and IterableDataset ]
                sampler = _InfiniteConstantSampler()
            else:  # map-style         # 如果dataset是map-style类型的（一般为此类型）
                if shuffle:
                    sampler = RandomSampler(dataset, generator=generator) # 设置shuffle，则随机传数据（randomsampler源码用randperm生成随机索引）
                else:
                    sampler = SequentialSampler(dataset)  # 不设置shuffle，则按顺序传数据
                    
            ...... # 后面内容省略     
                    
                   
        if batch_size is not None and batch_sampler is None:  # 如果设置了batch_size而没有单独设置batch_sampler
            # auto_collation without custom batch_sampler
            batch_sampler = BatchSampler(sampler, batch_size, drop_last) # 根据sampler中返回对元素索引以及batch_size大小生成每个batch
            
            ...... # 后面内容省略  
            
            
        if collate_fn is None:   # 如果没设置collate_fn
            if self._auto_collation:  # _auto_collation根据batch_sampler设置，如果batch_sampler是none，则返回false，而上面设置batch_size后会默认设置batch_sampler，则此处一般为True 
                collate_fn = _utils.collate.default_collate  # 输入batch，输出batch，里面有一些数据类型的转换，不改变batch顺序之类的信息
            else:
                collate_fn = _utils.collate.default_convert
                
            ...... # 后面内容省略  
            
            
            
    def _get_iterator(self) -> '_BaseDataLoaderIter':
        if self.num_workers == 0:
            return _SingleProcessDataLoaderIter(self)
        else:
            self.check_worker_number_rationality()
            return _MultiProcessingDataLoaderIter(self)
    
        ...... # 后面内容省略 
        
    
    def __iter__(self) -> '_BaseDataLoaderIter':  # 为dataloader对象定义iter方法，可以将其设置为一个迭代器：如iter(train_dataloader)，可用next方法输出一个batch
        # When using a single worker the returned iterator should be
        # created everytime to avoid reseting its state
        # However, in the case of a multiple workers iterator
        # the iterator is only created once in the lifetime of the
        # DataLoader object so that workers can be reused
        if self.persistent_workers and self.num_workers > 0:
            if self._iterator is None:
                self._iterator = self._get_iterator() 
            else:
                self._iterator._reset(self)
            return self._iterator
        else:
            return self._get_iterator() # 返回的是上面定义的get_iterator方法，可以根据上面设置的batch_sampler生成的索引输出一个batch的数据




### 总结
&emsp;&emsp;在处理数据集的过程中，首先需要构建一个dataset，dataset中进行数据有关的预处理，自定义dataset需实现***\_\_init__***，***\_\_len__***，***\_\_get_item__***方法。  

&emsp;&emsp;Dataloader类中定义了很多方法，如***sampler***定义了如何从dataset中取每个batch的顺序，根据相应顺序生成索引，如果设置了***shuffle=True***，则***sampler***会被系统自动赋值为***RandomSampler***类的对象，随机取数据，***batch_sampler***根据sampler中返回的元素索引以及batch_size大小生成batch,且可以设置***drop_last***设置是否丢弃最后一个不完整的batch，***collate_fn***可以将由***batch_sampler***生成的batch做后处理，如做序列填充，如果不对***collate_fn***额外设置，则由系统默认设置为***_utils.collate.default_collate***，在其中只对数据类型做一些判断和相应的转换工作。 

&emsp;&emsp;此外，由于dataloader类中定义了一个***\_\_iter__***方法，可以通过***iter***方法将dataloader对象转化为一个迭代器，源码在***\_\_iter__***方法中调用了之前定义的***self._get_iterator()***方法，该方法本质上是返回了***_SingleProcessDataLoaderIter()***类的对象，该类所继承的基类有***\_\_next__***方法，因此可以用**next(iter(train_dataset))**返回一个batch的数据。